In [ ]:
import numpy as np
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as ssim
import sys


ds_train, ds_val, ds_test = tfds.load('eurosat/all',
                               split=['train[:70%]', 'train[70%:90%]', 'train[90%:]'],
                               as_supervised = True)
train = ds_train.map(lambda image, label: image)
val = ds_val.map(lambda image, label: image)
test = ds_test.map(lambda image, label: image)
train
train_np = tfds.as_numpy(train)
val_np = tfds.as_numpy(val)
test_np = tfds.as_numpy(test)
train_np
x_train = []
x_val = []
x_test = []

for ex in train_np:
  #print (ex.shape)
  x_train.append(ex)

for ex in val_np:
  #print (ex.shape)
  x_val.append(ex)

for ex in test_np:
  #print (ex.shape)
  x_test.append(ex)


x_train = np.array(x_train)
x_val = np.array(x_val)
x_test = np.array(x_test)

x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
rows, cols, channels = ex.shape
print(ex.shape)


#%%
# Hybrid parallel (CNN-LSTM) Autoencoder archeticture
#rows, cols, channels = 64,64,13
input_img = keras.Input(shape=(rows, cols, channels))

###############################################################################
#sepctral encoder path
forwardSpectralFlattened = layers.Flatten()(input_img)
forwardSpectral = layers.Reshape([64,rows*channels])(forwardSpectralFlattened)
forwardSpectral = layers.LSTM(100, return_sequences=True)(forwardSpectral)
forwardSpectral = layers.LSTM(100, return_sequences=True)(forwardSpectral)
forwardSpectral = layers.LSTM(192, return_sequences=True)(forwardSpectral)
forwardSpectral = layers.LSTM(rows*channels, return_sequences=True)(forwardSpectral)
forwardSpectral = layers.Reshape([rows, cols, channels])(forwardSpectral)
forwardSpectral = layers.Conv2D(7, (3, 3), activation='PReLU', padding='same')(forwardSpectral)
print("forwardSpectral", forwardSpectral.shape)

##################################################################
#spatial encoder path for spatial feature extraction
forwardspatial = input_img
print ("\nforwardspatial before: ", forwardspatial.shape)
spatial_encoder = layers.Conv2D(256, (3, 3), activation='PReLU', padding='same')(forwardspatial)
forwardspatial = layers.BatchNormalization()(spatial_encoder)
spatial_encoder = layers.Conv2D(128, (3, 3), activation='PReLU', padding='same')(forwardspatial)
forwardspatial = layers.BatchNormalization()(spatial_encoder)
forwardspatial = layers.Conv2D(64, (3, 3), activation='PReLU', padding='same')(forwardspatial)
forwardspatial = layers.BatchNormalization()(forwardspatial)
forwardspatial = layers.Conv2D(32, (3, 3), activation='PReLU', padding='same')(forwardspatial)
forwardspatial = layers.BatchNormalization()(forwardspatial)
forwardspatial = layers.Conv2D(6, (3, 3), activation='PReLU', padding='same')(forwardspatial)
print ("forwardspatial after: ", forwardspatial.shape)

###############################################################
#Concatenate
Concatenated = layers.Concatenate(axis = 3)([forwardSpectral, forwardspatial])
print ("\nConcatenated : ",Concatenated.shape)

###############################################################################
#encoder Path cont.
encoded = layers.Conv2D(128, (3, 3), activation='PReLU', padding='same')(Concatenated)
encoded = layers.MaxPooling2D((2, 2), padding='same')(encoded)

encoded = layers.Conv2D(64, (3, 3), activation='PReLU', padding='same')(encoded)
encoded = layers.MaxPooling2D((2, 2), padding='same')(encoded)

encoded = layers.Conv2D(32, (3, 3), activation='PReLU', padding='same')(encoded)
encoded = layers.MaxPooling2D((2, 2), padding='same')(encoded)

###############################################################################
#decoder Path

decoded = layers.Conv2D(32, (3, 3), activation='PReLU', padding='same')(encoded)
decoded = layers.UpSampling2D((2, 2))(decoded)

decoded = layers.Conv2D(64, (3, 3), activation='PReLU', padding='same')(decoded)
decoded = layers.UpSampling2D((2, 2))(decoded)

decoded = layers.Conv2D(128, (3, 3), activation='PReLU', padding='same')(decoded)
decoded = layers.UpSampling2D((2, 2))(decoded)
decoded = layers.Conv2D(13, (3, 3), activation='PReLU', padding='same')(decoded)
print ("decoded : ",decoded.shape)


#################################
#sepctral decoder path
backwardSpectralFlattened = layers.Flatten()(decoded)
backwardSpectral = layers.Reshape([64,int(rows*channels)])(backwardSpectralFlattened)
backwardSpectral = layers.LSTM(192, return_sequences=True)(backwardSpectral)
backwardSpectral = layers.LSTM(100, return_sequences=True)(backwardSpectral)
backwardSpectral = layers.LSTM(100, return_sequences=True)(backwardSpectral)
backwardSpectral = layers.LSTM(rows*channels, return_sequences=True)(backwardSpectral)
backwardSpectral = layers.Reshape([rows, cols, channels])(backwardSpectral)
backwardSpectral = layers.Conv2D(7, (3, 3), activation='PReLU', padding='same')(backwardSpectral)
print ("\nbackwardSpectral : ",backwardSpectral.shape)

##################################################################
#spatial decoder path
backwardspatial = decoded
print ("\nbackwardSacial before: ",backwardspatial.shape)
backwardspatial = layers.Conv2D(32, (3, 3), activation='PReLU', padding='same')(backwardspatial)
backwardspatial = layers.BatchNormalization()(backwardspatial)
backwardspatial = layers.Conv2D(64, (3, 3), activation='PReLU', padding='same')(backwardspatial)
backwardspatial = layers.BatchNormalization()(backwardspatial)
backwardspatial = layers.Conv2D(128, (3, 3), activation='PReLU', padding='same')(backwardspatial)
backwardspatial = layers.BatchNormalization()(backwardspatial)
backwardspatial = layers.Conv2D(256, (3, 3), activation='PReLU', padding='same')(backwardspatial)
backwardspatial = layers.BatchNormalization()(backwardspatial)
backwardspatial = layers.Conv2D(6, (3, 3), activation='PReLU', padding='same')(backwardspatial)
print ("backwardSacial after: ",backwardspatial.shape)

output = layers.Concatenate(axis = 3)([backwardSpectral, backwardspatial])
print("\noutput: ", output.shape)

autoencoder = keras.Model(input_img, output)

autoencoder.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0001),
                    loss='mse', metrics=['accuracy'])
#autoencoder.compile(optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-06),
                   # loss='BinaryCrossentropy', metrics=['accuracy'])
autoencoder.summary()


history = autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=32,
                shuffle=True,
                validation_data=(x_val, x_val),
                #verbose = 1,
                #callbacks=[tensorboard_callback]
                )


# Import the 'os' module
import os

# Mount Google Drive
from google.colab import drive

drive.mount('/content/gdrive')
# Create the directory if it doesn't exist
weights_dir = '/content/gdrive/MyDrive/badr12_weights'  # Replace 'path/to/directory' with the desired directory path
if not os.path.exists(weights_dir):
    os.makedirs(weights_dir)

# Save the weights to the specified directory in the Spider environment
weights_path = os.path.join(weights_dir, 'hybrid_depth_one_spatial_parallel_multispectral_weights.h5')
autoencoder.save_weights(weights_path)



%tensorboard --logdir logs/fit

tf.keras.utils.plot_model(autoencoder, show_shapes=True)

plt.figure(1)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("MSE")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(['train','val'], loc = 'upper right')
plt.show

plt.figure(2)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model accuracy")
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(['train','val'], loc = 'lower right')
plt.show

reconstructed = autoencoder.predict(x_test)

from keras.utils.vis_utils import plot_model
plot_model(autoencoder, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

import numpy as np
from tensorflow.keras.preprocessing import image

############ CALULATE THE INFERENCE TIME ##############
# Load the image
img = autoencoder.predict(np.expand_dims(x_test[0], axis=0)) #REPLACE WITH LOADED MODEL NAME
reconstructed = []

_,r,c,ch = img.shape
img = np.reshape(img, (r,c,ch))
reconstructed.append(img)
reconstructed_array = np.array(reconstructed)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize if necessary


import time

start_time = time.time()
predictions = autoencoder.predict(img_array)
end_time = time.time()

inference_time = end_time - start_time
print(f"Inference time: {inference_time:.4f} seconds")


# Calculate FLOPs for a Conv2D layer
def calculate_conv2d_flops(layer):
    kernel_height, kernel_width = layer.kernel_size
    in_channels = layer.input_shape[-1]
    out_channels = layer.output_shape[-1]
    flops_per_element = kernel_height * kernel_width * in_channels
    flops_per_output = flops_per_element * out_channels
    total_flops = flops_per_output * np.prod(layer.output_shape[1:-1])
    return total_flops

# Calculate MACs for a Conv2D layer
def calculate_conv2d_macs(layer):
    total_flops = calculate_conv2d_flops(layer)
    return total_flops * 2  # Assuming one MAC for each FLOP

# Calculate FLOPs for an LSTM layer
def calculate_lstm_flops(layer):
    units = layer.units
    time_steps = layer.input_shape[1]
    flops_per_input = units * (2 * layer.input_shape[-1] + units - 1)
    total_flops = flops_per_input * time_steps
    return total_flops

# Calculate MACs for an LSTM layer
def calculate_lstm_macs(layer):
    total_flops = calculate_lstm_flops(layer)
    return total_flops * 2  # Assuming one MAC for each FLOP

# Calculate total FLOPs and MACs for the model
total_flops = 0
total_macs = 0

for layer in autoencoder.layers:
    if isinstance(layer, keras.layers.Conv2D):
        total_flops += calculate_conv2d_flops(layer)
        total_macs += calculate_conv2d_macs(layer)
    elif isinstance(layer, keras.layers.LSTM):
        total_flops += calculate_lstm_flops(layer)
        total_macs += calculate_lstm_macs(layer)

# Calculate computation time (randomly chosen value)
computation_time = 0.001  # 1 ms

# Calculate GFLOPS and GMAC
gflops = (total_flops / computation_time) * 1e-9
gmac = (total_macs / computation_time) * 1e-9

print("Total GFLOPS:", gflops)
print("Total GMAC:", gmac)

reconstructed = autoencoder.predict(x_test)



import numpy as np

def normalize_image(image):
    # Normalize the image to the range [0, 1]
    image = (image - image.min()) / (image.max() - image.min())
    return image
################################################################################
for i in range(0, 20):
    normalized_x_test = normalize_image(x_test[i])
    normalized_reconstructed = normalize_image(reconstructed[i])
    test_value = PSNR(normalized_x_test, normalized_reconstructed)
    print(f"PSNR test value no. {i+1} = {test_value} dB")
for i in range (0,20):
    ssim_skimage = ssim(
    normalized_x_test,
    normalized_reconstructed,
    data_range=x_test.max() - x_test.min(),
    channel_axis=-1)
    print(f"SSIM SKIMAGE no. {i+1} = {ssim_skimage}")
for i in range (0,20):
    mu_ssim_skimage = ssim(
    normalized_x_test,
    normalized_reconstructed,
    data_range=x_test.max() - x_test.min(),
    win_size=11,
    gaussian_weights=True,
    sigma=1.5,
    use_sample_covariance=False,
    channel_axis=-1)  # Specify the channel axis
    print(f"Multiscale SSIM SKIMAGE no. {i+1} = {mu_ssim_skimage}")
##################################################################
import numpy as np
from skimage.metrics import structural_similarity as ssim


# Load your original and reconstructed multispectral images.
# Ensure that they have the correct channel order and data range.

# Normalize the images to the desired range (e.g., [0, 1]).
normalized_original = (x_test[i] - x_test[i].min()) / (x_test[i].max() - x_test[i].min())
normalized_reconstructed = (reconstructed[i] - reconstructed[i].min()) / (reconstructed[i].max() - reconstructed[i].min())

# Initialize lists to store SSIM and MS-SSIM values for each channel.
ssim_values = []
ms_ssim_values = []

# Calculate SSIM and MS-SSIM for each channel.
for channel_idx in range(13):
    ssim_channel = ssim(
    normalized_original[..., 2],
    normalized_reconstructed[..., 2],
    data_range=x_test.max() - x_test.min(),
    channel_axis=-1)
    ms_ssim_channel = ssim(
    normalized_original[..., 2],
    normalized_reconstructed[..., 2],
    data_range=x_test.max() - x_test.min(),
    win_size=11,
    gaussian_weights=True,
    sigma=1.5,
    use_sample_covariance=False,
    channel_axis=-1)

    ssim_values.append(ssim_channel)
    ms_ssim_values.append(ms_ssim_channel)

# Calculate the overall SSIM and MS-SSIM values.
average_ssim = np.mean(ssim_values)
average_ms_ssim = np.mean(ms_ssim_values)

print(f"Overall SSIM: {average_ssim}")
print(f"Overall MS-SSIM: {average_ms_ssim}")
###########################################################################

# Initialize lists to store quality scores
psnr_scores = []
ssim_scores = []
mu_ssim_scores = []

for i in range(20):
    normalized_x_test = normalize_image(x_test[i])
    normalized_reconstructed = normalize_image(reconstructed[i])

    # Calculate PSNR for the pair of images
    psnr = PSNR(normalized_x_test, normalized_reconstructed)
    psnr_scores.append(psnr)

    # Calculate SSIM using skimage
    ssim_skimage = ssim(
    normalized_x_test,
    normalized_reconstructed,
    data_range=x_test.max() - x_test.min(),
    channel_axis=-1)
    ssim_scores.append(ssim_skimage)

    # Calculate Multiscale SSIM using skimage
    mu_ssim_skimage = ssim(
    normalized_x_test,
    normalized_reconstructed,
    data_range=x_test.max() - x_test.min(),
    win_size=11,
    gaussian_weights=True,
    sigma=1.5,
    use_sample_covariance=False,
    channel_axis=-1)
    mu_ssim_scores.append(mu_ssim_skimage)

# Calculate the average scores
average_psnr = np.mean(psnr_scores)
average_ssim = np.mean(ssim_scores)
average_mu_ssim = np.mean(mu_ssim_scores)

# Print the average scores
print(f"Average PSNR: {average_psnr} dB")
print(f"Average SSIM: {average_ssim}")
print(f"Average Multiscale SSIM: {average_mu_ssim}")

import matplotlib.pyplot as plt

for i in range(0,10):
    # Display grayscale images
    fig, axes = plt.subplots(1, 2, figsize=(5, 5))
    axes[0].imshow(x_test[i][:, :, 7], cmap='gray')
    axes[0].set_title(f"Original Image {i+1}")
    axes[1].imshow(reconstructed[i][:, :, 7], cmap='gray')
    axes[1].set_title(f"Reconstructed Image {i+1}")
    plt.show()

for i in range(0,5):
    # Display RGB composite images
    test_image = x_test[i][:, :, [1, 2, 3]]  # Create an RGB composite using bands 4, 2, and 1
    reconstructed_image = reconstructed[i][:, :, [1, 2, 3]]  # Create an RGB composite using bands 4, 2, and 1

    fig, axes = plt.subplots(1, 2, figsize=(5, 5))
    axes[0].imshow(test_image)
    axes[0].set_title(f"Original Image {i+1}")
    axes[1].imshow(reconstructed_image)
    axes[1].set_title(f"Reconstructed Image {i+1}")
    plt.show()

for i in range (0,10):
        fig = plt.figure(figsize=(4, 4))
        fig.add_subplot(1, 2,1)
        plt.imshow(x_test[i][:, :, 7], cmap='gray')
        fig.add_subplot(1, 2,2)
        plt.imshow(reconstructed[i][:, :, 7], cmap='gray')
        print(f"image no. {i+1} ")
        plt.show()

for i in range (0,5):
        fig = plt.figure(figsize=(5, 5))
        fig.add_subplot(1, 2,1)
        test_image = x_test[i][:, :, [3, 4, 5]]  # Create an RGB composite using bands 4, 2, and 1
        plt.imshow(test_image)
        fig.add_subplot(1, 2,2)
        reconstructed_image = reconstructed[i][:, :, [3, 4, 5]]  # Create an RGB composite using bands 4, 2, and 1
        plt.imshow(reconstructed_image)
        print(f"image no. {i+1} ")
        plt.show()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Training loop
epochs = 1  # Number of training epochs
batch_size = 64  # Batch size for training

# Create arrays to store loss values for plotting
history = {'loss': [], 'val_loss': []}

for epoch in range(epochs):
    # Shuffle the training data at the beginning of each epoch
    indices = np.arange(len(x_train))
    np.random.shuffle(indices)
    trainInput_shuffled = x_train[indices]

if (epoch + 1) % 1 == 0:  # Save and display images every epoch
        # Generate reconstructed images for the entire x_test
        reconstructed_images = autoencoder.predict(x_test, batch_size=16)

        # Create a grid of images (8x8) for both x_test and reconstructed images
        rows = 8
        cols = 8
        fig, axs = plt.subplots(rows*2, cols, figsize=(cols, rows*2))

        for i in range(rows):
            for j in range(cols):
                # Display original x_test image
                original_image = x_test[i * cols + j]  # Adjust shape as needed
                axs[i * 2, j].imshow(original_image[:, :, 7], cmap='gray')  # Display one channel for example
                axs[i * 2, j].axis('off')

                # Display corresponding reconstructed image
                reconstructed_image = reconstructed_images[i * cols + j]  # Adjust shape as needed
                axs[i * 2 + 1, j].imshow(reconstructed_image[:, :, 7], cmap='gray')  # Display one channel for example
                axs[i * 2 + 1, j].axis('off')

        plt.subplots_adjust(wspace=0.05, hspace=0.05)
        plt.suptitle(f'Original vs. Reconstructed Images (Epoch {epoch + 1})')
        plt.show()

In [ ]:
!pip install pydot graphviz
import pydot
from tensorflow.keras.utils import plot_model
plot_model(autoencoder, to_file='model.png', show_shapes=True, show_layer_names=True)
from IPython.display import Image
Image(filename='model.png')

In [ ]:
###################################### evaluation metrics (7) ############################
reconstructed = autoencoder.predict(x_test)
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as PSNR
from skimage.metrics import structural_similarity as SSIM
#from skimage.metrics import multichannel_ssim as MS_SSIM

# Initialize lists to store quality scores
psnr_scores = []
ssim_scores = []
ms_ssim_scores = []

band_to_evaluate = 7  # Band number you want to evaluate (0-indexed)

for i in range(20):  # Assuming you want to calculate scores for 20 images
    # Assuming x_test contains your original test images and reconstructed contains the reconstructed images
    original_band = x_test[i][:, :, band_to_evaluate]
    reconstructed_band = reconstructed[i][:, :, band_to_evaluate]

    # Calculate PSNR for the band
    psnr = PSNR(original_band, reconstructed_band)
    psnr_scores.append(psnr)

    # Calculate SSIM for the band
    ssim = SSIM(original_band, reconstructed_band, data_range=1.0)
    ssim_scores.append(ssim)

    # Calculate MS-SSIM for the band
    ms_ssim = SSIM(original_band, reconstructed_band, data_range=x_test.max() - x_test.min(), win_size=11, gaussian_weights=True, sigma=1.5, use_sample_covariance=False)
    ms_ssim_scores.append(ms_ssim)

# Calculate the average scores
average_psnr = np.mean(psnr_scores)
average_ssim = np.mean(ssim_scores)
average_ms_ssim = np.mean(ms_ssim_scores)

# Print the average scores
print(f"Average PSNR for Band {band_to_evaluate}: {average_psnr} dB")
print(f"Average SSIM for Band {band_to_evaluate}: {average_ssim}")
print(f"Average MS-SSIM for Band {band_to_evaluate}: {average_ms_ssim}")
################################################################################################